In [ ]:
library(Seurat)
library(SeuratDisk)
library(glue)
library(dplyr)
library(ggplot2)
library(SummarizedExperiment)
library(limma)
library(muscat)
library(purrr)

In [2]:
muscat_pbDS <- function(pb, sce, method) {
    res <- pbDS(pb, method=method, verbose=TRUE) 

    significant <- lapply(res$table[[1]], function(u) arrange(filter(u, p_adj.loc <= 0.05), p_adj.loc)) 
    all.sig.de <- bind_rows(significant)[-c(10)]

    n_de <- vapply(significant, nrow, numeric(1))
    p_de <- format(n_de / nrow(sce) * 100, digits = 3)
    print(data.frame("Num DE.genes" = n_de, "% DE.genes" = p_de, check.names = FALSE))

    return(bind_rows(res$table[[1]])[-c(10)])
}

In [7]:
muscat.de.genes <- function(seurat_obj, cluster_col, group_col, sample_col, method) {
    DefaultAssay(seurat_obj) <- "cellbender_corrected"
    cluster_ids <- unique(seurat_obj@meta.data[,cluster_col])
    
    sce <- SummarizedExperiment(
        assays=list(
            counts=seurat_obj@assays$cellbender_corrected@counts, 
            logcounts=seurat_obj@assays$cellbender_corrected@data
        ), 
        colData=seurat_obj@meta.data
    )
    sce <- as(sce, "SingleCellExperiment")
    sce <- prepSCE(sce, kid = cluster_col, gid = group_col, sid = sample_col, drop = TRUE)
    
    pb <- aggregateData(sce, assay = "counts", fun = "sum", by = c("cluster_id", "sample_id"))
    
    return(muscat_pbDS(pb, sce, method))
}

In [ ]:
all <- LoadH5Seurat("/home/dchafamo/final/data/combined_dataset_final_v2.h5seurat")

In [5]:
all@meta.data$cell_types_level_4[all@meta.data$cell_types_level_4=='Monocytes'] <- 'MonoMacs'
all@meta.data$cell_types_level_4[all@meta.data$cell_types_level_4=='Macrophages'] <- 'MonoMacs'

all@meta.data$cell_types_level_4[all@meta.data$cell_types_level_4=='Follicular_helper_T_cells'] <- 
    'Helper_Regulatory_T_cells'
all@meta.data$cell_types_level_4[all@meta.data$cell_types_level_4=='Regulatory_T_cells'] <- 
    'Helper_Regulatory_T_cells'

In [9]:
muscat.deseq2 <- muscat.de.genes(
    seurat_obj = all, 
    cluster_col = 'cell_types_level_4', 
    group_col = 'condition', 
    sample_col = 'donor', 
    method = 'DESeq2'
)

muscat.deseq2$logFC = muscat.deseq2$logFC * -1 # HL + on the right
# write.csv(muscat.deseq2, 'results/muscat_deseq2_level4_merged_RLNvsHL.csv')

  |======================================================================| 100%

                          Num DE.genes % DE.genes
B_GC                               403     1.3530
B_GC_cycling                       828     2.7798
B_memory                           601     2.0177
B_naive                            199     0.6681
BEC_arterial                        13     0.0436
BEC_venous                          42     0.1410
CD4_T_naive                        540     1.8129
CD8_T_effector                     573     1.9237
CD8_T_naive                        116     0.3894
DC_cycling                         333     1.1180
DC1                                132     0.4432
DC2                                296     0.9938
DN_cytotoxic_T_cells               101     0.3391
FDC                                 70     0.2350
FRC                               1081     3.6292
Helper_Regulatory_T_cells         2358     7.9165
ILC3                               102     0.3424
LEC                

In [16]:
ctype = "FRC"

In [ ]:
figure.path <- "figures/DE"

for (ctype in c("FDC", "FRC", "CD4_T_naive", "CD8_T_naive", "CD8_T_effector", "T_cycling",
                "MonoMacs", "Helper_Regulatory_T_cells")) {
    muscat.deseq2.ctype <- muscat.deseq2 %>% 
        filter(cluster_id == ctype) %>% 
        select(gene, p_adj.loc, logFC)

    rownames(muscat.deseq2.ctype) <- muscat.deseq2.ctype$gene
    colnames(muscat.deseq2.ctype) <- c('gene', 'p_val_adj', 'avg_log2FC')

    muscat.deseq2.ctype <- muscat.deseq2.ctype %>% select(p_val_adj, avg_log2FC)

    options(repr.plot.width=8, repr.plot.height=8, repr.plot.res=200)
    p <- SCpubr::do_VolcanoPlot(sample = all,
                                de_genes = muscat.deseq2.ctype, pt.size=1,
                                pval_cutoff = 1e-2,
                                FC_cutoff = 1, n_genes = 20, order_tags_by='pvalue', 
                                plot.title=glue("DE Genes in {ctype}"))

    SCpubr::save_Plot(
        plot = p, 
        figure_path = figure.path, 
        file_name = glue("{ctype}_level4_DE"), 
        width = 8, 
        height = 8, 
        output_format="all"
    )
}

In [6]:
HL <- subset(all, condition=='HL')

In [8]:
muscat.deseq2.ebv <- muscat.de.genes(
    seurat_obj = HL, 
    cluster_col = 'cell_types_level_3', 
    group_col = 'ebv_status', 
    sample_col = 'donor',
    method = 'DESeq2'
)

write.csv(muscat.deseq2.ebv, 'results/muscat_deseq2_ebv_level3_RLNvsHL.csv') # EBV+ on right

  |======================================================================| 100%

             Num DE.genes % DE.genes
B_cells                11    0.03693
BEC                     8    0.02686
CD4_T_cells             3    0.01007
CD8_T_cells             0    0.00000
FDC                     0    0.00000
Fibroblasts             5    0.01679
LEC                     0    0.00000
Macrophages            30    0.10072
mDC                     1    0.00336
Monocytes               0    0.00000
NK_cells                0    0.00000
pDC                     3    0.01007
Plasma_cells            0    0.00000
T_other                 0    0.00000
Tumor                 446    1.49735


In [22]:
muscat.deseq2.ebv <- muscat.de.genes(
    seurat_obj = HL, 
    cluster_col = 'cell_types_level_4', 
    group_col = 'ebv_status', 
    sample_col = 'donor',
    method = 'DESeq2'
)

write.csv(muscat.deseq2.ebv, 'results/muscat_deseq2_ebv_level4_merged_RLNvsHL.csv') # EBV+ on right

  |======================================================================| 100%

                          Num DE.genes % DE.genes
B_GC                                10    0.03357
B_GC_cycling                        13    0.04364
B_memory                             0    0.00000
B_naive                              0    0.00000
BEC_arterial                         5    0.01679
BEC_venous                          32    0.10743
CD4_T_naive                          1    0.00336
CD8_T_effector                       0    0.00000
CD8_T_naive                          0    0.00000
DC_cycling                           0    0.00000
DC1                                  1    0.00336
DC2                                  7    0.02350
DN_cytotoxic_T_cells                 0    0.00000
FDC                                  0    0.00000
FRC                                  6    0.02014
FRC_cycling                          1    0.00336
Helper_Regulatory_T_cells            2    0.00671
ILC3               

In [23]:
figure.path <- "figures/DE_ebv"

for (ctype in c("Helper_Regulatory_T_cells", "MonoMacs")) {
    muscat.deseq2.ctype <- muscat.deseq2.ebv %>% 
        filter(cluster_id == ctype) %>% 
        select(gene, p_adj.loc, logFC)

    rownames(muscat.deseq2.ctype) <- muscat.deseq2.ctype$gene
    colnames(muscat.deseq2.ctype) <- c('gene', 'p_val_adj', 'avg_log2FC')

    muscat.deseq2.ctype <- muscat.deseq2.ctype %>% select(p_val_adj, avg_log2FC)

    options(repr.plot.width=8, repr.plot.height=8, repr.plot.res=200)
    p <- SCpubr::do_VolcanoPlot(sample = all,
                                de_genes = muscat.deseq2.ctype, pt.size=1,
                                pval_cutoff = 1e-2,
                                FC_cutoff = 1, n_genes = 20, order_tags_by='pvalue', 
                                plot.title=glue("DE Genes in {ctype}"))

    SCpubr::save_Plot(
        plot = p, 
        figure_path = figure.path, 
        file_name = glue("{ctype}_level4_DE_ebv"), 
        width = 8, 
        height = 8, 
        output_format="all"
    )
}

Warning message:
“Removed 58 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 58 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 58 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 58 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 58 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 58 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 58 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 58 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 58 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 58 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 2077 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 2077 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 2077 rows containing m

In [ ]:
# Set necessary enrichR global options. This is copied from EnrichR code to avoid having to load the package.
suppressMessages({
  options(enrichR.base.address = "https://maayanlab.cloud/Enrichr/")
  options(enrichR.live = TRUE)
  options(modEnrichR.use = TRUE)
  options(enrichR.sites.base.address = "https://maayanlab.cloud/")
  options(enrichR.sites = c("Enrichr", "FlyEnrichr", "WormEnrichr", "YeastEnrichr", "FishEnrichr"))

  # Set the search to Human genes.
  enrichR::setEnrichrSite(site = "Enrichr")

  websiteLive <- TRUE
  dbs <- enrichR::listEnrichrDbs()
  # Get all the possible databases to query.
  dbs <- sort(dbs$libraryName)
})

# Choose the dataset to query against.
dbs_use <- c("GO_Biological_Process_2021")

In [ ]:
genes <- df_list$X16
enriched_terms <- enrichR::enrichr(genes, dbs_use)

options(repr.plot.width=14, repr.plot.height=14)
p <- SCpubr::do_TermEnrichmentPlot(enriched_terms = enriched_terms)
p